In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from gensim.models.word2vec import Word2Vec
from sklearn.naive_bayes import BernoulliNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
data = pd.read_csv("C:\\Users\\LENOVO\\Desktop\\train.csv")
data.head(5)

ID                                             Review  Rating
0   0  exceptional service nice all-around daughter s...       5
1   1  beautiful relaxing jw marriott desert ridge re...       5
2   2  great location great location 5 mins subway ta...       5
3   3  pleased nice safe hotel, flower market hotel v...       3
4   4  excellent hotel service great hotel excellent ...       4

In [4]:

# Libraries for text preprocessing
import re
import nltk

#nltk.download('stopwords')
from nltk.corpus import stopwords # import stopwords 

#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer # to lemmatize the words

from nltk.tokenize import RegexpTokenizer, word_tokenize # to tokenize sentences into words# TEXT NORMALIZATION   or PRE-PROCESSING

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
new_words = ['n',"th","hotel",'hotels','room','rooms','restaurant','restaurants','e','ca','nd','wo','el','etc']
stop_words = stop_words.union(new_words)

corpus = []
for index, row in data.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', row['Review']) # Remove punctuations    
    text = text.lower() #Convert to lowercase
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)  # remove tags
    text = re.sub("(\\d|\\W)+"," ",text)  # remove special characters and digits
    text = text.split()  # Convert to list from string
    
    #Lemmatisation
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if not word  in stop_words] 
    text_string = " ".join(text)
    data.at[index, 'clean_review'] = text_string
data[0:10]

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   
3   3  pleased nice safe hotel, flower market hotel v...       3   
4   4  excellent hotel service great hotel excellent ...       4   
5   5  beautiful beautiful beautiful, hotel just perf...       5   
6   6  not enought spend night business trip, 5 stars...       3   
7   7  great place, villa ixora good quiet location, ...       5   
8   8  march 14 28th 2007 wonderful relaxing vacation...       4   
9   9  fantastic value true gem stayed hotel husbands...       5   

                                        clean_review  
0  exceptional service nice around daughter staye...  
1  beautiful relaxing jw marriott desert ridge re...  
2  great location great location min subway take ...  
3  pleased nice safe flower market vast array mai...  
4  excellent service great excellent location cou...  
5  beautiful beautiful beautiful perfect thi perf...  
6  enought spend night business trip star buildin...  
7  great place villa ixora good quiet location wa...  
8  march wonderful relaxing vacation stayed catal...  
9  fantastic value true gem stayed husband birthd...

In [5]:
# tokenize the reviews

corpus = data["clean_review"].apply(word_tokenize)

In [6]:
corpus[0]

['exceptional',
 'service',
 'nice',
 'around',
 'daughter',
 'stayed',
 'well',
 'priced',
 'night',
 'king',
 'suite',
 'weekend',
 'night',
 'really',
 'impressed',
 'exceptionally',
 'welcoming',
 'service',
 'time',
 'reservation',
 'departure',
 'employee',
 'talked',
 'really',
 'friendly',
 'eager',
 'help',
 'question',
 'amazed',
 'day',
 'age',
 'l',
 'shaped',
 'pool',
 'tiny',
 'looked',
 'like',
 'fun',
 'kid',
 'hot',
 'tub',
 'exercise',
 'well',
 'breakfast',
 'abundant',
 'make',
 'waffle',
 'add',
 'scrambled',
 'egg',
 'bacon',
 'sausage',
 'fruit',
 'available',
 'bagel',
 'muffin',
 'oatmeal',
 'yogurt',
 'wanted',
 'morning',
 'sort',
 'long',
 'line',
 'waiting',
 'make',
 'waffle',
 'waiting',
 'scrambled',
 'egg',
 'picky',
 'really',
 'nice',
 'asked',
 'away',
 'freeway',
 'reading',
 'review',
 'earlier',
 'bit',
 'noisy',
 'hey',
 'city',
 'felt',
 'like',
 'deluxe',
 'considering',
 'reasonable',
 'price',
 'nice',
 'bedding',
 'art',
 'wall',
 'really',


In [7]:
len(corpus)

14343

In [8]:
y = data.Rating

In [9]:
# Splitting dataset in 70:30 ratio
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=0.30,random_state=123,stratify=y)

# Using Word2vec

In [10]:

w2v_model = Word2Vec(corpus,size=128,min_count=3,seed=1234,iter=10)
# extract word embeddings and average on documents
embeddings = w2v_model.wv
vocabulary = set(embeddings.vocab.keys())
train_doc2vec = np.array([np.mean([embeddings[word] for word in sent if word in vocabulary],axis=0) for sent in X_train])
test_doc2vec = np.array([np.mean([embeddings[word] for word in sent if word in vocabulary],axis=0) for sent in X_test])

In [11]:
words=w2v_model.wv.vocab
words

{'exceptional': <gensim.models.keyedvectors.Vocab at 0x1fa5940a0c8>,
 'service': <gensim.models.keyedvectors.Vocab at 0x1fa4ce83948>,
 'nice': <gensim.models.keyedvectors.Vocab at 0x1fa59430c08>,
 'around': <gensim.models.keyedvectors.Vocab at 0x1fa594240c8>,
 'daughter': <gensim.models.keyedvectors.Vocab at 0x1fa59424148>,
 'stayed': <gensim.models.keyedvectors.Vocab at 0x1fa594241c8>,
 'well': <gensim.models.keyedvectors.Vocab at 0x1fa59424208>,
 'priced': <gensim.models.keyedvectors.Vocab at 0x1fa59424248>,
 'night': <gensim.models.keyedvectors.Vocab at 0x1fa59424108>,
 'king': <gensim.models.keyedvectors.Vocab at 0x1fa59424188>,
 'suite': <gensim.models.keyedvectors.Vocab at 0x1fa59424288>,
 'weekend': <gensim.models.keyedvectors.Vocab at 0x1fa594242c8>,
 'really': <gensim.models.keyedvectors.Vocab at 0x1fa59424308>,
 'impressed': <gensim.models.keyedvectors.Vocab at 0x1fa59424348>,
 'exceptionally': <gensim.models.keyedvectors.Vocab at 0x1fa59424388>,
 'welcoming': <gensim.models.

In [12]:
vector=w2v_model.wv['agent']
vector

array([ 0.6249804 ,  0.3252056 , -0.22952867,  0.08600396, -0.7234352 ,
        1.1219963 ,  0.14342849, -1.0582557 , -1.6892729 , -0.20395295,
        0.34993833,  0.7504001 , -0.41388938,  0.00911111, -0.25592792,
        0.7577813 , -0.63264316, -1.226531  , -1.079811  , -0.69997823,
       -1.8862581 , -0.5627604 ,  0.2107288 ,  1.1758453 ,  0.9652073 ,
        0.6104646 ,  0.19597846, -0.31801373,  2.0831802 , -0.08807345,
        0.44474822,  1.1921254 ,  1.6421801 , -0.26939663,  0.18386135,
       -0.6723942 , -0.9792329 , -0.4604664 ,  0.41207132,  0.8536576 ,
        1.1673822 ,  2.1717923 , -1.1666702 , -0.02705524,  2.029392  ,
        2.810523  , -0.0040227 ,  0.5698561 , -2.5931535 ,  1.1981288 ,
        0.0735603 ,  1.186684  ,  0.6458477 ,  0.07542657,  2.0391345 ,
       -0.5188049 ,  1.304419  , -0.3234233 , -0.2223528 , -0.33672315,
       -0.14500086,  1.3378922 ,  0.32432154,  1.7490788 ,  0.57134175,
        3.2411306 ,  1.5754795 ,  1.4928125 ,  2.3391674 , -1.53

In [13]:
len(words)

14514

Logistic Regression using word2vec

In [14]:
clf_lr = LogisticRegression(max_iter=300)
clf_lr.fit(train_doc2vec,y_train)
train_acc = clf_lr.score(train_doc2vec,y_train)
test_acc = clf_lr.score(test_doc2vec,y_test)
print("Train accuracy is {}".format(train_acc))
print("Test accuracy is {}".format(test_acc))


Train accuracy is 0.6320717131474104
Test accuracy is 0.6153846153846154


C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
y_pred = clf_lr.predict(test_doc2vec)
y_pred

array([4, 5, 5, ..., 4, 4, 5], dtype=int64)

Random forest using word2vec

In [16]:
clf_rf = RandomForestClassifier(max_depth=12)
clf_rf.fit(train_doc2vec,y_train)
train_acc = clf_rf.score(train_doc2vec,y_train)
test_acc = clf_rf.score(test_doc2vec,y_test)
print("Train accuracy is {}".format(train_acc))
print("Test accuracy is {}".format(test_acc))

Train accuracy is 0.9608565737051793
Test accuracy is 0.5612363467348361


In [17]:
y_train.value_counts()

5    4505
4    2920
3    1057
2     874
1     684
Name: Rating, dtype: int64

# Using SMOTE, oversampled the data

In [18]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

X_sm, y_sm = smote.fit_resample(train_doc2vec,y_train)
X_test_sm, y_test_sm = smote.fit_resample(test_doc2vec,y_test)



In [19]:
y_sm.value_counts()

5    4505
4    4505
3    4505
2    4505
1    4505
Name: Rating, dtype: int64

In [20]:
clf_lr_sm = LogisticRegression(max_iter=300)
clf_lr_sm.fit(X_sm,y_sm)
train_acc_lr = clf_lr_sm.score(X_sm,y_sm)
test_acc_lr = clf_lr_sm.score(X_test_sm,y_test_sm)
print("Train accuracy is {}".format(train_acc_lr))
print("Test accuracy is {}".format(test_acc_lr))

Train accuracy is 0.630943396226415
Test accuracy is 0.5809425168306577


C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [21]:
clf_rf_sm = RandomForestClassifier(max_depth=12)
clf_rf_sm.fit(X_sm,y_sm)
train_acc_rf = clf_rf_sm.score(X_sm,y_sm)
test_acc_rf = clf_rf_sm.score(X_test_sm,y_test_sm)
print("Train accuracy is {}".format(train_acc_rf))
print("Test accuracy is {}".format(test_acc_rf))

Train accuracy is 0.9835294117647059
Test accuracy is 0.5410668047643707


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import math

In [ ]:
C = [10**x for x in range(-5,5)]
tuned_parameters = [{'C': C}]
model = GridSearchCV(LogisticRegression(class_weight='balanced'), tuned_parameters, scoring = 'accuracy', cv=3,return_train_score = True)
model.fit(train_doc2vec, y_train);

In [ ]:
from matplotlib import pyplot as plt

print(model.best_estimator_)
results = pd.DataFrame.from_dict(model.cv_results_)
results = results.sort_values(['param_C'])

train_score= results['mean_train_score']
train_acc_std= results['std_train_score']
cv_score = results['mean_test_score'] 
cv_acc_std= results['std_test_score']
C =  results['param_C']

C = [math.log(x,10) for x in C]
plt.plot(C, train_score, label='Train')

plt.plot(C, cv_acc_std, label='CV')

plt.scatter(C, train_score, label='Train points')
plt.scatter(C, cv_acc_std, label='CV points')

plt.legend()
plt.xlabel("C: hyperparameter")
plt.ylabel("Acc")
plt.grid()
plt.show()

In [ ]:

clf_lr_tuned = LogisticRegression(C=10000, class_weight='balanced')
clf_lr_tuned.fit(train_doc2vec,y_train)
train_acc_t = clf_lr_tuned.score(train_doc2vec,y_train)
test_acc_t = clf_lr_tuned.score(test_doc2vec,y_test)
print("Train accuracy is {}".format(train_acc_t))
print("Test accuracy is {}".format(test_acc_t))


In [24]:
X_full_train = corpus
y_full_train = data["Rating"]

In [25]:
w2v_model = Word2Vec(X_full_train,size=128,min_count=3,seed=1234,iter=10)
# extract word embeddings and average on documents
embeddings = w2v_model.wv
vocabulary = set(embeddings.vocab.keys())
train_doc2vec = np.array([np.mean([embeddings[word] for word in sent if word in vocabulary],axis=0) for sent in X_full_train])

In [26]:
clf_lr_f = LogisticRegression(max_iter=300)
clf_lr_f.fit(train_doc2vec,y_full_train)
train_acc = clf_lr.score(train_doc2vec,y_full_train)
print("Train accuracy is {}".format(train_acc))


Train accuracy is 0.5609705082618699


C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
y_pred = clf_lr.predict(test_doc2vec)
y_pred

In [ ]:
test_data=pd.read_csv("C:\\Users\\LENOVO\\Desktop\\test.csv")

In [ ]:
test_data.head()

In [ ]:
# Libraries for text preprocessing
import re
import nltk

#nltk.download('stopwords')
from nltk.corpus import stopwords # import stopwords 

#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer # to lemmatize the words

from nltk.tokenize import RegexpTokenizer, word_tokenize # to tokenize sentences into words# TEXT NORMALIZATION   or PRE-PROCESSING

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
new_words = ['n',"th","hotel",'hotels','room','rooms','restaurant','restaurants','e','ca','nd','wo','el','etc']
stop_words = stop_words.union(new_words)

corpus_test = []
for index, row in test_data.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', row['Review']) # Remove punctuations    
    text = text.lower() #Convert to lowercase
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)  # remove tags
    text = re.sub("(\\d|\\W)+"," ",text)  # remove special characters and digits
    text = text.split()  # Convert to list from string
    
    #Lemmatisation
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if not word  in stop_words] 
    text_string = " ".join(text)
    test_data.at[index, 'clean_review'] = text_string
test_data[0:10]

In [ ]:
corpus_test = test_data["clean_review"].apply(word_tokenize)

In [ ]:
len(corpus_test)

In [ ]:
X_full_test = corpus_test

In [ ]:
w2v_model_test = Word2Vec(X_full_test,size=128,min_count=3,seed=1234,iter=10)
# extract word embeddings and average on documents
embeddings = w2v_model_test.wv
vocabulary = set(embeddings.vocab.keys())
test_doc2vec_full = np.array([np.mean([embeddings[word] for word in sent if word in vocabulary],axis=0) for sent in X_full_test])

In [ ]:
y_pred_full = clf_lr_f.predict(test_doc2vec_full)
y_pred_full

In [ ]:
len(y_pred_full)

In [ ]:
type(y_pred_full)

In [ ]:
df = pd.DataFrame(y_pred_full, columns=['Rating'])
df.head()